In [1]:
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold
from sklearn import metrics
from contextlib import redirect_stdout
import scipy.sparse as sparse
import lightgbm as lgb
import pandas as pd
import numpy as np
import tqdm
import os
import gc
import sys
sys.path.append('../code/pipeline/')
sys.path.append('../code/utils/')
sys.path.append('../code/')
import data_pipeline as dp
import eval_utils as eu
import data_utils as du
import perf_utils as pu
import io_utils as iu
import config

In [2]:
# load multiple data and stack them together
dm = dp.DataManager(config.INPUT_DIR)
bin_loader = dm.build_data("raw", "binary")
lrstack_loader = dm.build_data("stacking", "lrCrossBinary_v2")
union_loader = dp.DataUnion(bin_loader, lrstack_loader)

with pu.profiler("loading training data"):
    cols_train, X_tv = union_loader.load("train")
    X_tv = sparse.csr_matrix(X_tv)
    gc.collect()

print("Train Data Shape: {}".format(X_tv.shape))
print("Train Column Numbers: {}".format(len(cols_train)))

[03:53:52] Finish loading training data. △M: +5.97GB. △T: 45.6 seconds.
Train Data Shape: (8798814, 419713)
Train Column Numbers: 419713


In [3]:
df_train = du.load_raw_data("train")
y = df_train['label'].values.copy()
y = (y + 1) / 2  # -1, 1 -> 0, 1

In [4]:
n_splits = 3
sss = StratifiedShuffleSplit(n_splits=n_splits, test_size=1 / 3, random_state=20180505)  # for reproducibility
split_indices = [(train_index, valid_index) for train_index, valid_index in sss.split(df_train, y)]

# n_splits = 3  # use 3 instead of 5 to save time
# skf = StratifiedKFold(n_splits=n_splits)
# split_indices = [(train_index, valid_index) for train_index, valid_index in skf.split(df_train, y)]

In [5]:
aids = df_train['aid'].values
with pu.profiler("splitting train/valid set"):
    train_index, valid_index = split_indices[0]
    X_train, X_valid = X_tv[train_index, :], X_tv[valid_index, :]
    y_train, y_valid = y[train_index], y[valid_index]
    aids_train, aids_valid = aids[train_index], aids[valid_index]
    assert X_train.shape[0] + X_valid.shape[0] == X_tv.shape[0]
    
    del X_tv
    gc.collect()
    
print("Training Set Size: {}".format(X_train.shape))
print("Validation Set Size: {}".format(X_valid.shape))

[03:54:31] Finish splitting train/valid set. △M: +124.68MB. △T: 28.6 seconds.
Training Set Size: (5865876, 419713)
Validation Set Size: (2932938, 419713)


In [6]:
with pu.profiler("preparing LightGBM data"):
    lgb_train = lgb.Dataset(X_train.astype(np.float32), y_train, feature_name=cols_train)
    lgb_valid = lgb.Dataset(X_valid.astype(np.float32), y_valid, feature_name=cols_train)
    gc.collect()

[03:54:35] Finish preparing LightGBM data. △M: +5.93GB. △T: 4.6 seconds.


In [7]:
log_folder = os.path.join(config.LOG_DIR, 'lgbm/pipeline/0515/')
log_file = 'v2.log'
log_path = os.path.join(log_folder, log_file)
os.makedirs(log_folder, exist_ok=True)

In [ ]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'max_depth': 15,
    'num_leaves': 120,
    'learning_rate': 0.15,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'verbose': 0
}
num_rounds = 500
eval_auc = eu.build_lightgbm_online_auc_eval(aids_train, aids_valid)

with iu.DuplicatedLogger(log_path):
    lgbm = lgb.train(params,
                     lgb_train,
                     num_boost_round=num_rounds,
                     valid_sets=[lgb_train, lgb_valid], 
                     valid_names=['train', 'valid1'],
                     feval = eval_auc,
                     early_stopping_rounds=50)

[1]	train's auc: 0.705385	train's online_auc: 0.702266	valid1's auc: 0.703766	valid1's online_auc: 0.698879
Training until validation scores don't improve for 50 rounds.
[2]	train's auc: 0.711111	train's online_auc: 0.70763	valid1's auc: 0.709097	valid1's online_auc: 0.704209
[3]	train's auc: 0.712792	train's online_auc: 0.709276	valid1's auc: 0.710421	valid1's online_auc: 0.705408
[4]	train's auc: 0.714158	train's online_auc: 0.711676	valid1's auc: 0.711778	valid1's online_auc: 0.708038
[5]	train's auc: 0.715209	train's online_auc: 0.712419	valid1's auc: 0.712807	valid1's online_auc: 0.708591
[6]	train's auc: 0.716259	train's online_auc: 0.71303	valid1's auc: 0.713789	valid1's online_auc: 0.708998
[7]	train's auc: 0.717036	train's online_auc: 0.713671	valid1's auc: 0.714589	valid1's online_auc: 0.709728
[8]	train's auc: 0.717706	train's online_auc: 0.714199	valid1's auc: 0.715154	valid1's online_auc: 0.710112
[9]	train's auc: 0.71827	train's online_auc: 0.715189	valid1's auc: 0.715701

In [9]:
# log_file = 'v2.feature_importance.csv'
# log_path = os.path.join(log_folder, log_file)

# df_feature_importance = pd.DataFrame({"feature": cols_train, "importance": lgbm.feature_importance()})
# df_feature_importance = df_feature_importance.sort_values("importance", ascending=False)
# df_feature_importance.to_csv(log_path, index=False)
# df_feature_importance.head(30)

In [13]:
# df_valid = df_train.iloc[valid_index]
proba_valid = lgbm.predict(X_valid)
df_score = eu.online_auc(aids_valid, y_valid, proba_valid, ret_verbose=True)
df_score = df_score.sort_values("auc", ascending=False)
df_score

,aid,auc
161,2054,0.936592
126,1672,0.932723
44,613,0.930529
39,519,0.929706
54,725,0.916698
149,1957,0.914612
48,671,0.911197
27,336,0.907132
145,1930,0.906762
35,450,0.905588


In [14]:
online_auc = df_score['auc'].mean()
simple_auc = metrics.roc_auc_score(y_valid, proba_valid)
print("Online AUC: {:.6f}".format(online_auc))
print("Simple AUC: {:.6f}".format(simple_auc))

Online AUC: 0.730226
Simple AUC: 0.735670


In [15]:
log_file = 'v2.online_auc.csv'
log_path = os.path.join(log_folder, log_file)
df_score.rename(columns={'selector': 'aid'}, inplace=True)
df_score = df_score[['aid', 'auc']]  # sort columns
df_score.to_csv(log_path, index=False)

In [16]:
with pu.profiler("cleaning memory"):
    del lgb_train
    del lgb_valid
    del X_train
    del X_valid
    gc.collect()

[03:40:03] Finish cleaning memory. △M: -11.97GB. △T: 1.4 seconds.


In [17]:
with pu.profiler("loading testing data"):
    cols_test, X_test = union_loader.load("test1")
    X_test = sparse.csr_matrix(X_test)
    gc.collect()

print("Test Data Shape: {}".format(X_test.shape))
print("Test Column Numbers: {}".format(len(cols_test)))

[03:40:26] Finish loading testing data. △M: +1.41GB. △T: 8.4 seconds.
Train Data Shape: (2265989, 419705)
Train Column Numbers: 419705


In [18]:
df_test = du.load_raw_data("test")
X_test = X_test.astype(np.float32)

with pu.profiler("making prediction on testing set"):
    proba_test = lgbm.predict(X_test)
    assert len(proba_test.shape) == 1
    assert proba_test.shape[0] == df_test.shape[0]

[03:40:50] Finish making prediction on testing set. △M: +0B. △T: 23.3 seconds.


In [19]:
subm_folder = '../subm/lgbm/0515_v2'
subm_file = 'submission.csv'
subm_path = os.path.join(subm_folder, subm_file)
os.makedirs(subm_folder, exist_ok=True)

subm = df_test.copy()
subm["score"] = proba_test
subm.to_csv(subm_path, index=False)